In [ ]:

# Install pyngrok for Colab if necessary
!pip install pyngrok

# Import ngrok and set authentication token for user's account
from pyngrok import ngrok
ngrok.set_auth_token("YOUR_NGROK_AUTH_TOKEN_HERE")  # Replace with the actual auth token

# Connect ngrok and print URL
public_url = ngrok.connect(5000)
print("Public ngrok URL:", public_url)
print("Remember to update this URL in your Spring Boot application each time you restart ngrok.")


In [1]:
from tensorflow.keras import backend as K
K.clear_session()

In [2]:
# Cell 1: Install Required Libraries
!pip install ultralytics  # YOLOv8
!pip install flask flask-ngrok
!pip install opencv-python-headless

In [3]:
# Cell 2: Import Libraries
import cv2
import numpy as np
from flask import Flask, jsonify, request
from google.colab.patches import cv2_imshow  # For displaying frames in Colab
# Ensure pyngrok is installed
!pip install pyngrok

# YOLOv8 Library
from ultralytics import YOLO

# Initialize Flask app
app = Flask(__name__)


In [4]:

# Cell 3: Load YOLOv8 Model
def load_yolov8_model():
    # Initialize YOLOv8 model (e.g., use YOLOv8n for smaller model)
    model = YOLO('yolov8n.pt')  # Change to 'yolov8l.pt' for larger, more accurate models if needed
    return model

yolo_model = load_yolov8_model()


In [5]:
# Refine `is_lane_violation` and `is_illegal_parking` functions with example logic

# Updated function for `detect_violations` to handle potential traffic violations
def detect_violations(frame, yolo_model):
    try:
        # Run YOLOv8 model on the frame
        results = yolo_model(frame)

        # Extract detections
        boxes, confidences, class_ids = [], [], []
        for detection in results[0].boxes:
            box = detection.xyxy[0].tolist()  # Bounding box [x1, y1, x2, y2]
            score = detection.conf[0]  # Confidence score
            cls_id = detection.cls[0]  # Class ID

            # Filter detections if required, for example, by score or class_id
            if score > 0.5:  # Only consider detections with confidence > 50%
                boxes.append([int(box[0]), int(box[1]), int(box[2]-box[0]), int(box[3]-box[1])])
                confidences.append(float(score))
                class_ids.append(int(cls_id))

        return boxes, confidences, class_ids
    except Exception as e:
        print(f"Error in detect_violations: {e}")
        return [], [], []


# Example logic for lane violation detection based on bounding box location
def is_lane_violation(box, lane_boundaries):
    x, y, w, h = box
    box_center_x = x + w // 2
    # Check if the vehicle center is outside the lane boundary (simplified logic)
    if not (lane_boundaries[0] <= box_center_x <= lane_boundaries[1]):
        return True
    return False


# Example logic for illegal parking detection
def is_illegal_parking(box, restricted_zone):
    x, y, w, h = box
    # Calculate the overlap area between the box and restricted zone
    box_area = max(0, min(x + w, restricted_zone[0] + restricted_zone[2]) - max(x, restricted_zone[0])) * \
               max(0, min(y + h, restricted_zone[1] + restricted_zone[3]) - max(y, restricted_zone[1]))
    restricted_zone_area = restricted_zone[2] * restricted_zone[3]

    # Violation if the overlap area covers a significant portion of the restricted zone
    if box_area / restricted_zone_area > 0.5:
        return True
    return False


In [6]:
# Cell 5: Analyze Violations Function
def analyze_violations(image_data):
    try:
        # Convert image data to an OpenCV frame
        np_arr = np.frombuffer(image_data, np.uint8)
        frame = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

        # Display frame in Colab to verify it's loading correctly
        cv2_imshow(frame)

        if frame is None:
            return {"error": "Invalid image data"}

        # Detect violations
        boxes, confidences, class_ids = detect_violations(frame)

        violations = []
        for i, box in enumerate(boxes):
            violation_type = None

            if is_lane_violation(box, frame):
                violation_type = "Lane Violation"
            elif is_illegal_parking(box, frame):
                violation_type = "Illegal Parking"

            if violation_type:
                violations.append({
                    "violation_type": violation_type,
                    "bounding_box": box,
                    "confidence": confidences[i],
                    "class_id": class_ids[i]
                })

        return {
            "violations_detected": len(violations),
            "violations": violations
        }
    except Exception as e:
        return {"error": "Internal server error", "message": str(e)}


In [7]:

from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route('/analyze', methods=['POST'])
def analyze():
    if 'file' not in request.files:
        print("No file part in the request")
        return "No file part", 400
    
    file = request.files['file']
    
    if file.filename == '':
        print("No selected file")
        return "No selected file", 400

    # Debugging output
    print("Received file:", file.filename)
    print("File content type:", file.content_type)

    # Process the file (add your detection logic here)
    response = {"violation": True}  # Mocked response for testing

    # Send response
    return jsonify(response)
            

In [ ]:

# Run Flask app - Run this cell to start server once after other setups are done
if __name__ == '__main__':
    app.run(port=5000)
            

ngrok tunnel URL: NgrokTunnel: "https://cf2e-35-236-189-122.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 17:09:48] "POST /analyze HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 17:09:48] "POST /analyze HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 17:09:48] "POST /analyze HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 17:09:48] "POST /analyze HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 17:09:48] "POST /analyze HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 17:09:48] "POST /analyze HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 17:09:48] "POST /analyze HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 17:09:49] "POST /analyze HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 17:09:49] "POST /analyze HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 17:09:49] "POST /